# **1. Creating a model with Single GRU layer for 450 research paper abstracts (for testing)**

### 1.1 Import libraries and required datasets(450 AI/ML research papers)

In [1]:
%load_ext tensorboard

In [1]:
from transformers import TFAutoModelForCausalLM, AutoTokenizer, AdamWeightDecay, pipeline, create_optimizer
from datasets import Dataset, DatasetDict, load_dataset
from transformers import DefaultDataCollator
import plotly.express as px
import plotly.io as pio
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import string
import re
import math
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pio.renderers.default = 'notebook_connected'

c:\Users\gupta\anaconda3\envs\virtualenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print("Tensorflow Version{}".format(tf.__version__))

Tensorflow Version2.11.0


In [4]:
data = r'arxivData.txt'

In [5]:
#Load the text file
text = open(data,'rb').read().decode(encoding='utf-8')
print('Dataset contains the total of {} characters'.format(len(text)))

Dataset contains the total of 64838939 characters


### 1.2 Preprocessing

Unique Characters

In [8]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

197 unique characters


Character to integer Mapping

In [9]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('{:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('}')

{
'\n':   0,
'\r':   1,
' ' :   2,
'!' :   3,
'"' :   4,
'#' :   5,
'$' :   6,
'%' :   7,
'&' :   8,
"'" :   9,
'(' :  10,
')' :  11,
'*' :  12,
'+' :  13,
',' :  14,
'-' :  15,
'.' :  16,
'/' :  17,
'0' :  18,
'1' :  19,
}


In [11]:
print('{}---char2int--- {}'.format(repr(text[38:63]), text_as_int[38:63]))

'itle,year\r\n"[{\'name\': \'Ah'---char2int--- [75 86 78 71 14 91 71 67 84  1  0  4 61 93  9 80 67 79 71  9 28  2  9 35
 74]


We are not doing stemming and lemmatization because we need letters and special characters such as for raised to ^ sign.

### 1.3 Prepare Dataset

We levarage a sliding window approach to train our model. We first set the maximum sequence length to 120 characters. This is done for the purpose of preparing and training batches.

In [12]:
#Maximum length of a sentence we want for a single input in characters
seq_length = 120
examples_per_epoch = len(text)//(seq_length+1)

#Create training examples
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

a
u
t
h
o
r
,
d
a
y


In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)#?
for item in sequences.take(10):
    print(repr(''.join(idx2char[item.numpy()])))
    print("-"*110)

'author,day,id,link,month,summary,tag,title,year\r\n"[{\'name\': \'Ahmed Osman\'}, {\'name\': \'Wojciech Samek\'}]",1,1802.00209v1,"'
--------------------------------------------------------------------------------------------------------------
"[{'rel': 'alternate', 'href': 'http://arxiv.org/abs/1802.00209v1', 'type': 'text/html'}, {'rel': 'related', 'href': 'http"
--------------------------------------------------------------------------------------------------------------
'://arxiv.org/pdf/1802.00209v1\', \'type\': \'application/pdf\', \'title\': \'pdf\'}]",2,"We propose an architecture for VQA which '
--------------------------------------------------------------------------------------------------------------
'utilizes recurrent layers to\ngenerate visual and textual attention. The memory characteristic of the\nproposed recurrent a'
--------------------------------------------------------------------------------------------------------------
'ttention units offers a rich joint emb

In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'author,day,id,link,month,summary,tag,title,year\r\n"[{\'name\': \'Ahmed Osman\'}, {\'name\': \'Wojciech Samek\'}]",1,1802.00209v1,'
Target data:  'uthor,day,id,link,month,summary,tag,title,year\r\n"[{\'name\': \'Ahmed Osman\'}, {\'name\': \'Wojciech Samek\'}]",1,1802.00209v1,"'


In [16]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print(" input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print(" expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
 input: 67 ('a')
 expected output: 87 ('u')
Step    1
 input: 87 ('u')
 expected output: 86 ('t')
Step    2
 input: 86 ('t')
 expected output: 74 ('h')
Step    3
 input: 74 ('h')
 expected output: 81 ('o')
Step    4
 input: 81 ('o')
 expected output: 84 ('r')


In [17]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

In [18]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print("Dataset Shape={}".format(dataset))

Dataset Shape=<BatchDataset element_spec=(TensorSpec(shape=(64, 120), dtype=tf.int32, name=None), TensorSpec(shape=(64, 120), dtype=tf.int32, name=None))>


In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    """
    Utility to create model object
    Parameters:
        vocab_size: number of unique characters
        embedding_dim: size of embedding vector. This is basically in power of 2
        rnn_units: number if GRU units to be used
        batch_size: batch size for training model.
    Returns:
        tf.keras model object
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)#softmax?
    ])
    return model

In [20]:
#Lenth of the vocabulary in chars
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

In [21]:
model = build_model(vocab_size = vocab_size, embedding_dim = embedding_dim, rnn_units = rnn_units, batch_size=BATCH_SIZE)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           50432     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 197)           201925    
                                                                 
Total params: 4,190,661
Trainable params: 4,190,661
Non-trainable params: 0
_________________________________________________________________


In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [24]:
model.compile(optimizer='adam',loss=loss)

In [25]:
checkpoint_dir = r'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [26]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
EPOCHS = 64
hostory = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback,tensorboard_callback])

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'training_checkpoints\\ckpt_256'

### 1.4 Build model using the function created above with the parameters

Using model.summary we saw how the batch size shows up as one of the parametres which determined shape.
For inference, we would be using a single input sentence/context to generate text.Thus we build the model again using buil_model utility we preprared earlier but use a batch_size of 1 this time. Once we have the model object with desired batch size, we use load_weights to utilize the latest checkpoint weights for instance. 

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 256)            20480     
                                                                 
 gru_3 (GRU)                 (1, None, 1024)           3938304   
                                                                 
 dense_3 (Dense)             (1, None, 80)             82000     
                                                                 
Total params: 4,040,784
Trainable params: 4,040,784
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, context_string, num_generate=1000, temperature=1.0):
    input_eval = [char2idx[s] for s in context_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions,0)

        predictions = predictions / temperature

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (context_string + ''.join(text_generated))

Temperature is scaling factor. It helps in controlling the output randomness of our model.

### 1.5 Test model's text generation using seeds

In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100))

polulation and samples are seberale, and e samplen and then in a cances of the sample is nemeat.

It is impation is the sama


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=2))

polulation and samples arequentiFy an3 (BuporiGable and siviomqAr)

When(0. 
o4xan8 and Bizh $V&,5q6 w5 Dtimptrin a spumait-


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=0.5))

polulation and samples are used at of s ald adults living in the sumbor of the semeat, a dataset when in a peprarente that the


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=0.1))

polulation and samples are used to represent the population is a bertain cance of the population is the sumber of observations


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=0.234))

polulation and samples are used to describe and summarize data.

The mean is a table or graph that che and to mean is a meas


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=0.32564))

polulation and samples are used to describe and summarize the semeat of the population is a table or grapustics of a sample of


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=0.01)) #Best result

polulation and samples are used to describe and summarize the population is the sumber of observations in the size of the popu


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=200,temperature=0.1))

polulation and samples are used to describe and summarize the population is a measure of the sample data. In the sample data. In the sample data. It is a way to measure the population is the sumber of observations in the size 


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=200,temperature=0.001))

polulation and samples are used to describe and summarize the population is the sumber of observations in the size of the population is the sumber of observations in the size of the population is the sumber of observations in 


# **2. Creating a model with single GRU layer for 25k research paper abstracts**

### 2.1 Loading data (25k papers)

In [ ]:
df = pd.read_csv('df_train.csv')
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,title,abstract
0,99520,99520,99520.0,Multiplayer Performative Prediction: Learning ...,Learning problems commonly exhibit an intere...
1,70875,70875,70875.0,Estimating Vector Fields from Noisy Time Series,While there has been a surge of recent inter...
2,69168,69168,69168.0,3D-OES: Viewpoint-Invariant Object-Factorized ...,We propose an action-conditioned dynamics mo...
3,11441,11441,11441.0,Canonical Correlation Analysis for Analyzing S...,We propose using canonical correlation analy...
4,31746,31746,31746.0,Reducing Noise in GAN Training with Variance R...,We study the effect of the stochastic gradie...


In [ ]:
# Removed all columns except abstract
df = df.drop(df.columns[[0, 1, 2, 3]], axis=1)
df.head()

,abstract
0,Learning problems commonly exhibit an intere...
1,While there has been a surge of recent inter...
2,We propose an action-conditioned dynamics mo...
3,We propose using canonical correlation analy...
4,We study the effect of the stochastic gradie...


In [ ]:
df.head()

,abstract
0,Learning problems commonly exhibit an intere...
1,While there has been a surge of recent inter...
2,We propose an action-conditioned dynamics mo...
3,We propose using canonical correlation analy...
4,We study the effect of the stochastic gradie...


### 2.2 Cleaning data

In [ ]:
def clean(text):
    text = re.sub('\n', ' ', text)
    text = re.sub('\'', '\'', text)
    text = re.sub('\s+', ' ', text)
    return text

In [ ]:
# Pre-processing
df['abstract'] = df['abstract'].map(lambda x: x[2: -1])
df['abstract'] = df['abstract'].map(lambda x: clean(x))
text = df.to_string(index=False)
text = clean(text)
text = text[text.find(df.iloc[0]['abstract'][0:3]):]

Printing unique characters

In [ ]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

Preparing vocabulary

In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('{:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('}')

{
' ' :   0,
'!' :   1,
'"' :   2,
'#' :   3,
'$' :   4,
'%' :   5,
'&' :   6,
"'" :   7,
'(' :   8,
')' :   9,
'*' :  10,
'+' :  11,
',' :  12,
'-' :  13,
'.' :  14,
'/' :  15,
'0' :  16,
'1' :  17,
'2' :  18,
'3' :  19,
}


In [ ]:
print('{}---char2int--- {}'.format(repr(text[38:63]), text_as_int[38:63]))

'interesting feedback mech'---char2int--- [73 78 84 69 82 69 83 84 73 78 71  0 70 69 69 68 66 65 67 75  0 77 69 67
 72]


Generating sequences

In [ ]:
#Maximum length of a sentence we want for a single input in characters
seq_length = 120
examples_per_epoch = len(text)//(seq_length+1)

#Create training examples
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

L
e
a
r
n
i
n
g
 
p


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(30):
    print(repr(''.join(idx2char[item.numpy()])))
    print("-"*110)

'Learning problems commonly exhibit an interesting feedback mechanism wherein the population data reacts to competing deci'
--------------------------------------------------------------------------------------------------------------
'sion makers\' actions. This paper formulates a new game theoretic framework for this phenomenon, called "multi-player perf'
--------------------------------------------------------------------------------------------------------------
'ormative prediction". We focus on two distinct solution concepts, namely (i) performatively stable equilibria and (ii) Na'
--------------------------------------------------------------------------------------------------------------
'sh equilibria of the game. The latter equilibria are arguably more informative, but can be found efficiently only when th'
--------------------------------------------------------------------------------------------------------------
'e game is monotone. We show that under mild assumptions, t

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Learning problems commonly exhibit an interesting feedback mechanism wherein the population data reacts to competing dec'
Target data:  'earning problems commonly exhibit an interesting feedback mechanism wherein the population data reacts to competing deci'


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print(" input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print(" expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
 input: 44 ('L')
 expected output: 69 ('e')
Step    1
 input: 69 ('e')
 expected output: 65 ('a')
Step    2
 input: 65 ('a')
 expected output: 82 ('r')
Step    3
 input: 82 ('r')
 expected output: 78 ('n')
Step    4
 input: 78 ('n')
 expected output: 73 ('i')


In [ ]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print("Dataset Shape={}".format(dataset))

Dataset Shape=<BatchDataset element_spec=(TensorSpec(shape=(64, 120), dtype=tf.int32, name=None), TensorSpec(shape=(64, 120), dtype=tf.int32, name=None))>


### 2.3 Build model based on the parameters specified below

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    """
    Utility to create model object
    Parameters:
        vocab_size: number of unique characters
        embedding_dim: size of embedding vector. This is basically in power of 2
        rnn_units: number if GRU units to be used
        batch_size: batch size for training model.
    Returns:
        tf.keras model object
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, 
            stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
#Lenth of the vocabulary in chars
vocab_size = len(vocab)

embedding_dim = 256

rnn_units = 1024

99


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           25344     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 99)            101475    
                                                                 
Total params: 4,065,123
Trainable params: 4,065,123
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss=loss)

In [ ]:
checkpoint_dir = r'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [ ]:
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard("logs/", histogram_freq=1)

In [ ]:
EPOCHS = 10
hostory = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback,tensorboard_callback])

Epoch 1/10
3737/3737 [==============================] - 268s 70ms/step - loss: 1.1827
Epoch 2/10
3737/3737 [==============================] - 310s 82ms/step - loss: 0.9765
Epoch 3/10
3737/3737 [==============================] - 334s 89ms/step - loss: 0.9539
Epoch 4/10
3737/3737 [==============================] - 357s 95ms/step - loss: 0.9467
Epoch 5/10
3737/3737 [==============================] - 355s 94ms/step - loss: 0.9468
Epoch 6/10
3737/3737 [==============================] - 353s 94ms/step - loss: 0.9528
Epoch 7/10
3737/3737 [==============================] - 356s 95ms/step - loss: 0.9731
Epoch 8/10
3737/3737 [==============================] - 353s 94ms/step - loss: 1.6183
Epoch 9/10
3737/3737 [==============================] - 357s 95ms/step - loss: 1.7201
Epoch 10/10
3737/3737 [==============================] - 356s 95ms/step - loss: 1.6246


### 2.4 Test model's text generation based on few seeds

In [ ]:
print(generate_text(model, context_string=u"Convolutional Neural Networks",num_generate=100))

Convolutional Neural Networks (ANN). It is the envaningly, deiffic Memony treandar invand boust-plaring has in hige satermane rit


In [ ]:
print(generate_text(model, context_string=u"Generative adversarial networks (GANs) have gained increasing popularity",num_generate=100,temperature=0.1))

Generative adversarial networks (GANs) have gained increasing popularity of the problem convex the model with the supportable that are complexity of the state-of-the-art mo


In [ ]:
print(generate_text(model, context_string=u"Generative adversarial networks (GANs) have gained increasing popularity",num_generate=100,temperature=0.5))

Generative adversarial networks (GANs) have gained increasing popularity of the patching the optimization are sublementations of and ther optification of learning models in


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=0.1))

The memory characteristic of the proposed recurrent attention units are for example of its efficiency. Moreover, different stages of MKRL can be seamlessly integrated into


# **3. Creating a model with single LSTM layer for 25k research paper abstracts**

### 3.1 Build model based on the layers specified below

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    """
    Utility to create model object
    Parameters:
        vocab_size: number of unique characters
        embedding_dim: size of embedding vector. This is basically in power of 2
        rnn_units: number if GRU units to be used
        batch_size: batch size for training model.
    Returns:
        tf.keras model object
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True),
        tf.keras.layers.Dense(vocab_size, activation='softmax')
    ])
    return model

In [ ]:
model = build_model(vocab_size = vocab_size, embedding_dim = embedding_dim, rnn_units = rnn_units, batch_size=BATCH_SIZE)

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           25344     
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_2 (Dense)             (64, None, 99)            101475    
                                                                 
Total params: 5,373,795
Trainable params: 5,373,795
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
# logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard("logs_SingleLSTMSoftMax/", histogram_freq=1)

In [ ]:
EPOCHS = 16
model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback,tensorboard_callback])

Epoch 1/16
3737/3737 [==============================] - 157s 41ms/step - loss: 1.1531 - accuracy: 0.6578
Epoch 2/16
3737/3737 [==============================] - 156s 42ms/step - loss: 0.9956 - accuracy: 0.6999
Epoch 3/16
3737/3737 [==============================] - 156s 41ms/step - loss: 0.9598 - accuracy: 0.7098
Epoch 4/16
3737/3737 [==============================] - 157s 42ms/step - loss: 0.9396 - accuracy: 0.7154
Epoch 5/16
3737/3737 [==============================] - 156s 42ms/step - loss: 0.9260 - accuracy: 0.7193
Epoch 6/16
3737/3737 [==============================] - 157s 42ms/step - loss: 0.9162 - accuracy: 0.7221
Epoch 7/16
3737/3737 [==============================] - 160s 42ms/step - loss: 0.9084 - accuracy: 0.7243
Epoch 8/16
3737/3737 [==============================] - 158s 42ms/step - loss: 0.9025 - accuracy: 0.7260
Epoch 9/16
3737/3737 [==============================] - 154s 41ms/step - loss: 0.8977 - accuracy: 0.7274
Epoch 10/16
3737/3737 [==============================] 

In [ ]:
EPOCHS = 32
model.fit(dataset, epochs=EPOCHS, initial_epoch=16,callbacks=[checkpoint_callback,tensorboard_callback])

Epoch 17/32
3737/3737 [==============================] - 155s 41ms/step - loss: 0.8781 - accuracy: 0.7331
Epoch 18/32
3737/3737 [==============================] - 157s 42ms/step - loss: 0.8769 - accuracy: 0.7334
Epoch 19/32
3737/3737 [==============================] - 158s 42ms/step - loss: 0.8760 - accuracy: 0.7337
Epoch 20/32
3737/3737 [==============================] - 159s 42ms/step - loss: 0.8751 - accuracy: 0.7340
Epoch 21/32
3737/3737 [==============================] - 158s 42ms/step - loss: 0.8744 - accuracy: 0.7342
Epoch 22/32
3737/3737 [==============================] - 162s 43ms/step - loss: 0.8739 - accuracy: 0.7343
Epoch 23/32
3737/3737 [==============================] - 160s 43ms/step - loss: 0.8732 - accuracy: 0.7345
Epoch 24/32
3737/3737 [==============================] - 157s 42ms/step - loss: 0.8729 - accuracy: 0.7346
Epoch 25/32
3737/3737 [==============================] - 155s 41ms/step - loss: 0.8725 - accuracy: 0.7347
Epoch 26/32
3737/3737 [=======================

### 3.2 Test model's text generation based on the seeds given below

In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=0.1))

The memory characteristic of the proposed recurrent attention units are for example of its efficiency. Moreover, different stages of MKRL can be seamlessly integrated into


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=0.234))

The memory characteristic of the proposed recurrent attention units are for example of its encrypted dataset to a semi-trusted cloud comparisons are paper, we present a ma


In [ ]:
print(generate_text(model, context_string=u"The memory characteristic of the proposed recurrent attention units are",num_generate=100,temperature=0.32564))

The memory characteristic of the proposed recurrent attention units are focused on the real-world data collected data containing the second positive resources spent in the


# **4. Creating a model using Generic transformer**

### 4.1 Import and install required libraries and pretrained models

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.5 MB/s eta 0:00:00


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
text = "Multisensory object-centric perception"
encoded_input = tokenizer.encode(text, return_tensors='pt')

In [ ]:
encoded_input

tensor([[15205,   271,   641,   652,  2134,    12, 28577, 11202]])

In [ ]:
tokenizer.decode(encoded_input[0][0])

'Mult'

### 4.2 Test model's text generation based on the seeds given 

In [ ]:
output = model.generate(encoded_input, max_length=200, num_beams=5, no_repeat_ngram_size=2, early_stopping=True) #Beam algorith to predict the next word # ngram is taking two two words as input

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tokenizer.decode(output[0], skip_special_tokens=True)

'Multisensory object-centric perception.\n\nIn this article, we will look at some of the most important aspects of visual perception and how they can be used to improve your perception of your surroundings. We will also discuss how you can use this knowledge to help you better understand the world around you.'

# **5. Creating a model using DistilGPT2 transformer (Transfer Learning Model)**

### 5.1 Import and install required libraries and pretrained model (DistilGPT-2)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
model = TFAutoModelForCausalLM.from_pretrained("distilgpt2", pad_token_id=tokenizer.eos_token_id)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


The dataset for the fine-tuning operation is available on the Huggingface Hub, and it’s a subset of a bigger dataset hosted on Kaggle.

The original dataset, published by Cornell University, contains titles and abstracts of 1.7M+ scientific papers belonging to the STEM category. The subset hosted on the Huggingface Hub contains information on around 100K papers pertaining to the machine learning category.

I decided to fine-tune DistilGPT-2 on abstracts only. I started by loading the dataset from the Huggingface Hub.

In [ ]:
data = load_dataset("CShorten/ML-ArXiv-Papers", split='train')
data

Using custom data configuration CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211
Found cached dataset csv (C:/Users/gupta/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'title', 'abstract'],
    num_rows: 117592
})

### 5.2 Preprocessing (Train-test split, tokenization)

In [ ]:
data = data.train_test_split(shuffle = True, seed = 200, test_size=0.2)

train = data["train"]
val = data["test"]

Loading cached split indices for dataset at C:\Users\gupta\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-5fcced655579dd43.arrow and C:\Users\gupta\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-10bb285727aaa68b.arrow


In [ ]:
# The tokenization function
def tokenization(data):
    tokens = tokenizer(data["abstract"], padding="max_length", truncation=True, max_length=300)
    return tokens

# Apply the tokenizer in batch mode and drop all the columns except the tokenization result
train_token = train.map(tokenization, batched = True, remove_columns=["title", "abstract", "Unnamed: 0", "Unnamed: 0.1"])
val_token = val.map(tokenization, batched = True, remove_columns=["title", "abstract", "Unnamed: 0", "Unnamed: 0.1"])


Loading cached processed dataset at C:\Users\gupta\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-cf1610aeae1e7e01.arrow
Loading cached processed dataset at C:\Users\gupta\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-43ea478b4774c95d.arrow


In [ ]:
# Create labels as a copy of input_ids
def create_labels(text):
    text["labels"] = text["input_ids"].copy()
    return text

# Add the labels column using map()
lm_train = train_token.map(create_labels, batched=True, num_proc=10)
lm_val = val_token.map(create_labels, batched=True, num_proc=10)

Loading cached processed dataset at C:\Users\gupta\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-342d0ad02f377fc9.arrow
Loading cached processed dataset at C:\Users\gupta\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-7792641a8a9d718a.arrow
Loading cached processed dataset at C:\Users\gupta\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-525c02bbc6880d8e.arrow
Loading cached processed dataset at C:\Users\gupta\.cache\huggingface\datasets\CShorten___csv\CShorten--ML-ArXiv-Papers-0dcddd7fc76c9211\0.0.0\6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317\cache-b46ee8c7167b3efe.arrow
Loading cached processed dataset at C:\Users\gupta\.

In [ ]:
train_set = model.prepare_tf_dataset(
    lm_train,
    shuffle=True,
    batch_size=8
)

validation_set = model.prepare_tf_dataset(
    lm_val,
    shuffle=False,
    batch_size=8
)

### 5.3 Compile the model using optimizers and our input dataset

In [ ]:
# Setting up the learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005,
    decay_steps=500,
    decay_rate=0.95,
    staircase=False)
    
# Exponential decay learning rate
optimizer = AdamWeightDecay(learning_rate=lr_schedule, weight_decay_rate=0.01)


In [ ]:
model.compile(optimizer=optimizer, metrics=['accuracy'])
model.summary()

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 81912576  
 r)                                                              
                                                                 
Total params: 81,912,576
Trainable params: 81,912,576
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Fit with callbacks
model.fit(train_set, validation_data=validation_set, epochs=1)

11759/11759 [==============================] - 3533s 300ms/step - loss: 2.4148 - accuracy: 0.2275 - val_loss: 2.2005 - val_accuracy: 0.2267


In [ ]:
model.save_weights('transformer.h5')

In [ ]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    framework="tf",
    max_new_tokens=500
)

### 5.4 Testing the model's text generation using variable input seeds

In [ ]:
test_sentence = "clustering"
text_generator(test_sentence)

c:\Users\gupta\anaconda3\envs\virtualenv\lib\site-packages\transformers\generation\tf_utils.py:603: UserWarning:

You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)



[{'generated_text': 'clustering of high-dimensional data requires data-driven models that\ncapture the intrinsic properties of data. To develop such models which are\nreliable in data due to their complex relationships to dimensionality, the\nalgorithm for unsupervised classification of high-dimensional data has\nrecently been proposed. The proposed approach relies on a novel deep\nconvolutional neural network to learn how to effectively learn to\nlearn from small number of observations, such as from a single low-level\nsensor for example. The proposed network combines a convolutional neural\nnetwork that learns to segment the data by introducing a graph with\ndifferent weights, as observed in image reconstruction. The results\nshow the superior performance of our approach using multiple benchmarks\nand image data sets with different structural properties, and an\napplication to an urban-scale data set. All code and data are available at\nhttps://github.com/lun/sustainablenetwork.\n'}]

In [ ]:
test_sentence = "The memory characteristic of the proposed recurrent attention units are"
text_generator(test_sentence)

[{'generated_text': "The memory characteristic of the proposed recurrent attention units are\nthat they are initialized to be connected to the same input sequence. The\nimportance of this type of mechanism in recurrent neural networks has been\nevaluated in terms of generalization error while it has been widely recognized\nthat it can be thought of as an important and necessary mechanism to achieve\nbetter learning performance than conventional recurrent attention units. In\nthis study, inspired by the idea of neural networks in the deep learning\ncommunity, we propose a novel RNN learning architecture based on a simple yet\nimportant feature of recurrent deep learning, named the residual attention unit. The\nproposed deep residual attention unit architecture comprises two modules,\nwhich consists of the fully connected LSTM-like layer with the attention unit\nlayers, the first module which produces a feature representation by a novel deep\ncontrastive divergence based on different rec

In [ ]:
test_sentence = "enchanced meta-heuristic (ML-ACO) that combines"
text_generator(test_sentence)

[{'generated_text': 'enchanced meta-heuristic (ML-ACO) that combines the multi-armed bandit\n(MAB) model to achieve improved return are a promising direction despite their\nhigh computational cost and computational time. However, it is still unclear\nwhether MAB in certain ML-ACO scenarios performs sufficiently well for\ngeneralized users and/or in specific ML applications. To solve this problem\nwe propose an ML-ACO model that is more general and versatile than the\nmulti-armed bandit bandit model (MAB). Our contribution is to derive a principled\nframework that incorporates both MAB and existing bandit algorithms with a\nnovel strategy to improve the performance of the performance of the\nmulti-armed online gradient descent algorithm at hand. Nested with the proposed\nalgorithms and experimental results, we demonstrate that our proposed\napproach outperforms all previous algorithms by significant margins.\n'}]

In [ ]:
test_sentence = "Deep reinforcement learning"
text_generator(test_sentence)

[{'generated_text': "Deep reinforcement learning has emerged as a powerful method for solving\nsequential control tasks. However, in some applications the number of trained\ndeep Q-networks and task-relevant features often remains a large challenge. For\ninstance, high performance deep reinforcement learning architectures like Montezuma's\nQ-learning often make it difficult to deploy such architectures on\nresource-poor systems like GPUs. These limitations complicate deep Q-network\nlearning, especially in the low resource regime which can achieve higher\nconvergence, e.g. from one GPU. One way to address these limitations is\nto train multi-layer Q-networks that are capable of efficiently learning joint\nstructures across different tasks, with high-quality outputs. We present\nan RL-based method to solve a variety of complex tasks in this low resource region\nthat can be considered a single Q-network with lower computational cost and\nperformance loss. Specifically, we investigate how